# **Branch Splitting and Geometric Analysis**
<hr>

In [ ]:
from vmtk import vmtkscripts
import numpy as np
import os

The default current directory when loading a notebook from GDrive is the home folder of the current user. To load the user's libraries and modules, first change to the file location using the os' python module, and load the necessary modules, then change to the former directory. 

In [ ]:
# Change to directory where vmtk_functions and vmtk_filenames are located
os.chdir('/home/iagolessa/Documents/aneurysms/vmtkScripts')

# Import the necessary user's modules
import vmtk_functions
from vmtk_filenames import *

# Change it back
os.chdir('/home/iagolessa')

In [ ]:
# Path to save the generated files when extracting the surface
# We reccomend to separate it from the DICOM original dir. 
# Note that inside casePath, this code will create different sub-directories to store different data types
casePath       = '/home/iagolessa/Documents/aneurysms/geometries/einsteinCases/unrupturedCases/vmtkReconstruction/case11/'

# Define subdirs
imagesDir       = casePath+'images/'
surfacesDir     = casePath+'surfaces/'
centerlinesDir  = casePath+'centerlines/'
meshesDir       = casePath+'meshes/'
parentVesselDir = casePath+"parentVessel/"

print('Files saved to:', casePath)

<hr/>
# **Branch Splitting**

The following cells demonstrates how to split a vascular segment into its constituent branches and, if it is the case, an aneurysm. This is specially good to extract the aneurysm surface using the centerline that can be constructed inside the aneurysm. Of course, since the tool is designed to split a vascular tree in its branches, it may not extract exactly the aneurysm surface correctly depending on the vascular and aneurysm geometry.

## Centerlines Splitting

In [ ]:
# Get array with pacthes info
surface = vmtk_functions.readSurface(surfacesDir+surfaceOrientedFile)
capsGeoInfo = vmtk_functions.getPatchInfo(surface)

In [ ]:
print(capsGeoInfo)

In [ ]:
# Clears object outletBarycenters if exists
outletBarycenters = None
inletBarycenters = []

# create tuples with inlet and outlet barycenters
for i in range( len(capsGeoInfo) ):
    if capsGeoInfo['PatchType'][i] == 'inlet':
        inletBarycenters.append(capsGeoInfo['Center'][i])
    else: 
        try:
            outletBarycenters += capsGeoInfo['Center'][i]
        except:
            outletBarycenters = capsGeoInfo['Center'][i]
            
inletBarycenters, outletBarycenters

In [ ]:
# Aneurysm top point to get aneurysm centerline
aneurysmTopPoint = (5.20944, -17.7144, 4.4689) # aneurysm location

In [ ]:
# Computing centerlines
# This code computes centerlines for a generic number of inlets
# It does that by computing the centerlines for each source barycenter
# and then appending the resulting centerlines with 'vmtksurfaceappend'

centerlinesList = []

for source in inletBarycenters:
    # Instantiate vmtkcenterline object
    centerlines = vmtkscripts.vmtkCenterlines()
    # Surface to be used
    centerlines.Surface = surface
    # Type of seed selector: by source coordinate
    centerlines.SeedSelectorName = 'pointlist'
    centerlines.SourcePoints = source
    centerlines.TargetPoints = list(outletBarycenters+aneurysmTopPoint)
    centerlines.CheckNonManifold = 1
    centerlines.AppendEndPoints = 1
    centerlines.Execute()
    centerlinesList.append(centerlines.Centerlines)

# Provide managing exception if there is only one centerline
centerlineMain = centerlinesList[0]

In [ ]:
# If more than one inlet, then the centerlines for 
# each inlet is appended here by a vmtk script

if len(centerlinesList) > 1:
    for centerline in centerlinesList[1:]:
        # Instantiate vmtksurfaceappend object
        centerlinesAppend = vmtkscripts.vmtkSurfaceAppend()
        centerlinesAppend.Surface = centerlineMain
        centerlinesAppend.Surface2 = centerline
        centerlinesAppend.Execute()
        # Store final centerlines to centerline main
        # therefore, centerlineMain will store the final complete centerline
        centerlineMain = centerlinesAppend.Surface
else:
    print('Only one source.')

# Centerline smoother
centerlineSmoother = vmtkscripts.vmtkCenterlineSmoothing()
centerlineSmoother.Centerlines = centerlineMain
centerlineSmoother.SurfaceOutputFileName = centerlinesDir+centerlineWithAneurysm
centerlineSmoother.Execute()
centerlineSmoother.IOWrite()

vmtk_functions.viewSurface(centerlineSmoother.Centerlines)

In [ ]:
# Compute Voronoi Diagram Separately for parent vessel reconstruction
voronoiDiagram = vmtkscripts.vmtkDelaunayVoronoi()

voronoiDiagram.Surface = surface
voronoiDiagram.CheckNonManifold = 1
voronoiDiagram.RemoveSubresolutionTetrahedra = 1
voronoiDiagram.VoronoiDiagramOutputFileName = surfacesDir+'VoronoiDiagram.vtp' #VoronoiDiagramFile

voronoiDiagram.Execute()
voronoiDiagram.IOWrite()

# Debug
# voronoiDiagram.PrintInputMembers()
# voronoiDiagram.PrintOutputMembers()
# vmtk_functions.viewSurface(voronoiDiagram.Surface)

In [ ]:
centerlineSection = vmtkscripts.vmtkCenterlineSections()

centerlineSection.Surface = surface
centerlineSection.Centerlines = centerlineSmoother.Centerlines

centerlineSection.CenterlinesOutputFileName = centerlinesDir+centerlineSectionsFile
centerlineSection.Execute()
centerlineSection.IOWrite()

# Debug
# centerlineSection.PrintInputMembers()
# centerlineSection.PrintOutputMembers()

In [ ]:
# Computing centerlines Frenet system
centerlineGeometry = vmtkscripts.vmtkCenterlineGeometry()

centerlineGeometry.Centerlines = centerlineSection.Centerlines
centerlineGeometry.CenterlinesOutputFileName = centerlinesDir+centerlineGeometryFile
centerlineGeometry.LineSmoothing = 0
centerlineGeometry.Execute()
centerlineGeometry.IOWrite()

# Debug
# centerlineGeometry.PrintInputMembers()
# centerlineGeometry.PrintOutputMembers()

## Centerline attributes

Centerlines are defined by the position of their points. In order to define the position of a point along the centerline or to quantify the angular position of a point on the surface around the centerline, it is necessary to equip centerlines with two attributes, Abscissas and Normals.

Abscissas are easy to define: they measure the distances along a line. The point with abscissa 0 is not necessarily the first point of the centerline, but can be chosen to represent a landmark along a centerline, such that, if we have a population of models, all the abscissas will be referred to the same anatomical location. In absence of such location, abscissas can be generated relative to the starting point and can be offset to a different location at a later time.

Parallel transport reference systems: given a starting reference system (the analogous to the 0 abscissa point), the next reference system is generated by moving along the centerline of a small amount and rotating the system on the osculating plane of an amount equal to the change in the orientation of the line tangent. This operation is repeated until all points are equipped with a reference system. Since reference systems are rotated within the osculating plane no artificial torsion is introduced proceeding along the centerline.

The script that computes abscissas and parallel transport normals is vmtkcenterlineattributes. It computes abscissas and normals relative to the first point of each centerline (they can be offset to landmarks at a later time). As such, it should be called before 'vmtkbranchextractor', otherwise each tract into which centerlines are split will have an abscissa starting from 0 at the first point.

In [ ]:
# Computation of centerlines attributes
# -- Based on the parallel transport theory
centerlineAttributes = vmtkscripts.vmtkCenterlineAttributes()

centerlineAttributes.CenterlinesOutputFileName = centerlinesDir+'centerlinesAttributes.vtp' #centerlinesAttributesFile
centerlineAttributes.Centerlines = centerlineGeometry.Centerlines

centerlineAttributes.Execute()
centerlineAttributes.IOWrite()

# Debug
# centerlineAttributes.PrintInputMembers()
# centerlineAttributes.PrintOutputMembers()

In [ ]:
# Extracting branches of centerlines
# this script splits the centerline in branches
# corresponding to the different tubes formed by the Maximum Inscribed Spheres
branchExtractor = vmtkscripts.vmtkBranchExtractor()

# If centerline is read from external file
# branchExtractor.CenterlinesInputFileName = casePath+'centerlines/centerlinesAttributes.vtp'
# branchExtractor.IORead()

# If calculated right above
branchExtractor.Centerlines = centerlineAttributes.Centerlines
branchExtractor.RadiusArrayName = 'MaximumInscribedSphereRadius'
branchExtractor.CenterlinesOutputFileName = centerlinesDir+'branchExtracted.vtp'

branchExtractor.Execute()
branchExtractor.IOWrite()

# Debug
# branchExtractor.PrintInputMembers()
branchExtractor.PrintOutputMembers()

In [ ]:
vmtk_functions.viewCenterline(branchExtractor.Centerlines, branchExtractor.GroupIdsArrayName)

## Splitting the vessels and aneurysm surface

Lets now move on to surface splitting. We now want to clip the original surface into its branches. We do it the following way using the vmtkbranchclipper script. Before using it, it is recommended to change the labels of the splitted branch to identify the aneurysm region, if needed.

In [ ]:
# Before splitting surface branches, if the aneurysm surface
# is needed, then it is good to relabel the group ids of the centerlines

# Changing labels of centerlines branches
# 0 -> 0
# 1 -> 0
# 2 -> 1
# 3 -> 1
# 4 -> 0
# 5 -> 0
# 6 -> 0
# 7 -> 0

# 8 -> 1
# 9 -> 1
# 10 -> 1
# 11 -> 1

centerlinesLabeler = vmtkscripts.vmtkCenterlineLabeler()

# The labeling process depends on the original ids 
# So it should be inspected by the user before execution
# so the Labeling attribute be updated accordingly 
centerlinesLabeler.Centerlines = branchExtractor.Centerlines
# Reading external file
# centerlinesLabeler.CenterlinesInputFileName = casePath+"centerlines/branchExtracted.vtp" 
# centerlinesLabeler.IORead()

# viewCenterline(centerlinesLabeler.Centerlines,'GroupIds')

# Parameters to save to disk
# centerlinesLabeler.CenterlinesOutputFileName = casePath+'centerlines/branchAneurysmExtracted.vtp'

# centerlinesLabeler.Centerlines = branchExtractor.Centerlines
centerlinesLabeler.GroupIdsArrayName = branchExtractor.GroupIdsArrayName
centerlinesLabeler.LabelIdsArrayName = branchExtractor.GroupIdsArrayName
centerlinesLabeler.Labeling = [0, 0,
                               1, 1,
                               2, 2,
                               3, 3,
                               4, 4]

centerlinesLabeler.PrintInputMembers()
centerlinesLabeler.PrintOutputMembers()

centerlinesLabeler.Execute()
# centerlinesLabeler.IOWrite()

In [ ]:
# Visualizing centerlines group ids after extraction
viewCenterline(centerlinesLabeler.Centerlines, centerlinesLabeler.LabelIdsArrayName)

In [ ]:
# Performing splitting and grouping on the surface
# based on centerlines splitting

# Input centerline to use: from branchExtractor directly 
# centerlinesToSplitSurface = branchExtractor.Centerlines
# groupIdsArray = branchExtractor.GroupIdsArrayName
# or from labeler
centerlinesToSplitSurface = centerlinesLabeler.Centerlines

# Branch Clipper
branchClipper = vmtkscripts.vmtkBranchClipper()

# Surface to clip
branchClipper.Surface = surfaceToMesh.Surface
# branchClipper.SurfaceInputFileName = casePath+'surfaces/patches/wall.stl' #surfaceRemeshedSm.vtp'
# branchClipper.IORead()

# Centerlines: if the file is loaded externally
# branchClipper.CenterlinesInputFileName = casePath+"centerlines/branchExtracted.vtp"
# branchClipper.IORead()

# or if it was computed here
branchClipper.Centerlines = centerlinesToSplitSurface

# Other parameters
branchClipper.RadiusArrayName   = 'MaximumInscribedSphereRadius'
branchClipper.GroupIdsArrayName = branchExtractor.GroupIdsArrayName # centerlinesLabeler.GroupIdsArrayName
branchClipper.BlankingArrayName = branchExtractor.BlankingArrayName
branchClipper.RadiusArrayName   = branchExtractor.RadiusArrayName

# If only some patches must be computed
# branchClipper.GroupIds = [0] 
# branchClipper.ClipValue = -3

# branchClipper.SurfaceOutputFileName = casePath+"surfaces/patches/aneurysm.stl"
# branchClipper.SurfaceOutputFileName = casePath+"surfaces/branchAneurysmClipped.vtp"

branchClipper.Execute()
# branchClipper.IOWrite()

# Debug
# branchClipper.PrintInputMembers()
branchClipper.PrintOutputMembers()

In [ ]:
# Visualization of the surface
surfaceViewer = vmtkscripts.vmtkSurfaceViewer()
surfaceViewer.Surface = branchClipper.Surface
surfaceViewer.ArrayName = branchClipper.GroupIdsArrayName
surfaceViewer.Legend = 1
surfaceViewer.Execute()

<hr/>
# **Geometrical Analysis**

This tutorial demonstrates how to analyze the 3D geometry of a vascular segment. Using vmtk, you should have been able to compute centerlines, to identify bifurcations and branches on the centerlines and to clip the surface according to the branches. Now weÃÂÃÂÃÂÃÂ¢ÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂll face the problem of quantifying geometric features of the vascular segment, those associated to bifurcations, such as bifurcation planes and bifurcation angles, and those associated to branches, such as curvature and torsion.

## Computing bifurcation reference systems 

We here consider a single bifurcation: the extension to a more general case is rather straightforward. From the centerline splitting and grouping phase we were left with a first group that ends with the second reference points (one for each tract), a second group representing the bifurcation whose tracts start with the second reference point and end with the first, and two last groups representing the daughter branches that start with the first reference points (where each centerline exits the other tube). We define the bifurcation reference system the following way:

--> The origin of the bifurcation is defined as the barycenter of the four reference points weighted by the surface of the maximum inscribed sphere defined on the reference points. The reason of the weighting is that small branches have less impact on the position of the bifurcation origin;

--> The normal to the bifurcation plane is defined as the normal to the polygon defined by the four reference points computed in the bifurcation origin;

--> The bifurcation upnormal is the similarly weighted average vector between the vectors pointing from the second to the first reference point on each centerline

In [ ]:
# Computing the bifurcation reference system
# it uses the output of Branch Extractor script because 
# it depends on array calculated there, especially 
# because it identifies the bifurcations and the tracts
bifurcationRefSystem = vmtkscripts.vmtkBifurcationReferenceSystems()

# inputFile = casePath+"centerlines/centerlines.vtp"
# bifurcationRefSystem.CenterlinesInputFileName = inputFile
# bifurcationRefSystem.IORead()

bifurcationRefSystem.Centerlines = branchExtractor.Centerlines
bifurcationRefSystem.RadiusArrayName = branchExtractor.RadiusArrayName
bifurcationRefSystem.BlankingArrayName = branchExtractor.BlankingArrayName
bifurcationRefSystem.GroupIdsArrayName = branchExtractor.GroupIdsArrayName

# bifurcationRefSystem.ReferenceSystemsOutputFileName = centerlinesDir+'bifurcationRefSystem.vtp'

bifurcationRefSystem.Execute()
# Writing to disk parameters
# bifurcationRefSystem.IOWrite()

# Debug
# bifurcationRefSystem.PrintInputMembers()
# bifurcationRefSystem.PrintOutputMembers()

## Offsetting centerline attributes to bifurcations

As anticipated, centerline attributes (i.e. abscissas and parallel transport normals), which were computed relative to the first point of the centerline (not a very meaningful landmark), can be offset to bifurcations. This way, a population of vessels will have the 0 abscissa corresponding to the origin of the same bifurcation, which sounds like a reasonable thing to do. Similarly, parallel transport normals at the bifurcation will be oriented like the bifurcation plane normal.

In [ ]:
# To offset the centerline attributes, we need to chose the bifurcation
# so inspecting centerlines

vmtk_functions.viewCenterline(branchExtractor.Centerlines, branchExtractor.GroupIdsArrayName)

In [ ]:
bifurcationId = 1

# Offsetting the centerlines attributes
centerlineOffsetAttributes = vmtkscripts.vmtkCenterlineOffsetAttributes()

# Input parameters 
centerlineOffsetAttributes.Centerlines = branchExtractor.Centerlines
centerlineOffsetAttributes.ReferenceSystems = bifurcationRefSystem.ReferenceSystems

centerlineOffsetAttributes.AbscissasArrayName = centerlineAttributes.AbscissasArrayName
centerlineOffsetAttributes.NormalsArrayName   = centerlineAttributes.NormalsArrayName
centerlineOffsetAttributes.GroupIdsArrayName  = branchExtractor.GroupIdsArrayName
centerlineOffsetAttributes.CenterlineIdsArrayName = branchExtractor.CenterlineIdsArrayName

normalsArrayName = bifurcationRefSystem.ReferenceSystemsNormalArrayName
centerlineOffsetAttributes.ReferenceSystemsNormalArrayName = normalsArrayName

# Bifurcation ID
centerlineOffsetAttributes.ReferenceGroupId = bifurcationId

# Writing parameters
# centerlineOffsetAttributes.CenterlinesOutputFileName = centerlinesDir+'bifurcationRefSystemOffset.vtp'

centerlineOffsetAttributes.Execute()

# Writing to disk parameters
# centerlineOffsetAttributes.IOWrite()

# Debug
# centerlineOffsetAttributes.PrintInputMembers()
centerlineOffsetAttributes.PrintOutputMembers()

## Computing bifurcation geometry

Each branch, parent or daughter, arrives or leaves the bifurcation region with a certain orientation. Such orientation can in general be decomposed in in-plane and out-of-plane components, where the plane is the bifurcation plane. The geometry of the bifurcation will therefore be described by the in-plane and out-of-plane angles that each branch forms with the bifurcation reference system. Taking the upnormal as an in-plane reference orientation, each branch at the bifurcation is described by its in-plane angle with the upnormal direction and by its out-of-plane angle with the bifurcation normal.

In [ ]:
# To compute bifurcation geometry we use the vmtkbifurcationvectors
bifurcationVectors = vmtkscripts.vmtkBifurcationVectors()

# Input parameters
bifurcationVectors.Centerlines = branchExtractor.Centerlines
 
bifurcationVectors.ReferenceSystems = bifurcationRefSystem.ReferenceSystems
bifurcationVectors.RadiusArrayName = branchExtractor.RadiusArrayName
bifurcationVectors.CenterlineIdsArrayName = branchExtractor.CenterlineIdsArrayName
bifurcationVectors.GroupIdsArrayName = branchExtractor.GroupIdsArrayName
bifurcationVectors.TractIdsArrayName = branchExtractor.TractIdsArrayName
bifurcationVectors.BlankingArrayName = branchExtractor.BlankingArrayName
bifurcationVectors.ReferenceSystemsNormalArrayName = bifurcationRefSystem.ReferenceSystemsNormalArrayName
bifurcationVectors.ReferenceSystemsUpNormalArrayName = bifurcationRefSystem.ReferenceSystemsUpNormalArrayName
# bifurcationVectors.BifurcationVectorsOutputFileName = centerlinesDir+'bifurcationVectors.vtp' 

bifurcationVectors.Execute()
# bifurcationVectors.IOWrite()

# Debug
# bifurcationVectors.PrintInputMembers()
bifurcationVectors.PrintOutputMembers()

In [ ]:
# Computing bifurcations sections 
bifurcationSections =  vmtkscripts.vmtkBifurcationSections()

# Surface and centerlines already split into branches
bifurcationSections.Surface= branchClipper.Surface
bifurcationSections.Centerlines = branchClipper.Centerlines
bifurcationSections.NumberOfDistanceSpheres = 0

# Arrays specifications
bifurcationSections.RadiusArrayName = branchExtractor.RadiusArrayName
bifurcationSections.CenterlineIdsArrayName = branchExtractor.CenterlineIdsArrayName
bifurcationSections.GroupIdsArrayName = branchExtractor.GroupIdsArrayName
bifurcationSections.TractIdsArrayName = branchExtractor.TractIdsArrayName
bifurcationSections.BlankingArrayName = branchExtractor.BlankingArrayName
bifurcationSections.BifurcationSectionsOutputFileName = centerlinesDir+'bifurcationSections0.vtp'

bifurcationSections.Execute()
bifurcationSections.IOWrite()

# Debug
# bifurcationSections.PrintInputMembers()
bifurcationSections.PrintOutputMembers()

In [ ]:
# Computing bifurcations profiles (curves on branch extractor joining locations)
bifurcationProfiles =  vmtkscripts.vmtkBifurcationProfiles()

# Surface and centerlines already split into branches
bifurcationProfiles.Surface = branchClipper.Surface
bifurcationProfiles.Centerlines = branchClipper.Centerlines

# Arrays specifications
bifurcationProfiles.RadiusArrayName = branchExtractor.RadiusArrayName
bifurcationProfiles.CenterlineIdsArrayName = branchExtractor.CenterlineIdsArrayName
bifurcationProfiles.GroupIdsArrayName = branchExtractor.GroupIdsArrayName
bifurcationProfiles.TractIdsArrayName = branchExtractor.TractIdsArrayName
bifurcationProfiles.BlankingArrayName = branchExtractor.BlankingArrayName

bifurcationProfiles.BifurcationProfilesOutputFileName = casePath+"centerlines/bifurcationProfiles.vtp"

bifurcationProfiles.Execute()
bifurcationProfiles.IOWrite()

# Debug
# bifurcationProfiles.PrintInputMembers()
bifurcationProfiles.PrintOutputMembers()